<a href="https://colab.research.google.com/github/goyalpramod/paper_implementations/blob/main/AI_agents_from_first_principles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is not like my usual paper implementations, but it talks about and shows how to build AI agents from the ground up and in simplified manner

Most of the code here has been inspired by the following two blogs:
* [OpenAI Orchestrating Agents](https://cookbook.openai.com/examples/orchestrating_agents)
* [Anthropic Building effective agents](https://www.anthropic.com/research/building-effective-agents)

## Setup LLM

I have decided to go with openai as I had some credits lying around, same thing can be done for anthropic or any other LLM provider. Make sure to check there docs out.


In [ ]:
!pip install openai

In [ ]:
import os

os.environ["OPENAI_API_KEY"]="your_api_key"

In [3]:
from pydantic import BaseModel
from typing import Optional, List
import json
import inspect


from openai import OpenAI

client = OpenAI()

In [4]:
def run_llm(content: str = None, messages: Optional[List[str]] = [], tool_schemas: Optional[List[str]] = [], system_message: str = "You are a helpful assistant."):
    # Build base request parameters
    request_params = {
        "model": "gpt-4o-mini",
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": content}
        ] + messages
    }

    # Only add tools parameter if tool_schemas is provided and non-empty
    if tool_schemas:
        request_params["tools"] = tool_schemas

    # Make the API call with conditional parameters
    completion = client.chat.completions.create(**request_params)

    response = completion.choices[0].message
    messages.append(response)

    return messages

In [5]:
# wont run with empty tools


run_llm(
    content = """["apple", "pie", 42, 2, 13]""",
    system_message = """
    You are an expert classifier, which classifies strings and integers.\
    Given a list of numbers and words, only return the numbers as a list.\
    You will be given the inputs inside <input> tags.\

    Input:  <input>["hello", 42, "pizza", 2, 5]</input>
    Output: [42,2,5]
    """,
    tool_schemas = None,
)

[ChatCompletionMessage(content='[42, 2, 13]', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)]

In [6]:
def function_to_schema(func) -> dict:
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
        },
    }

In [7]:
from typing import List, Union
import ast

def add_numbers(num_list: Union[List[int], str]) -> int:
    """
    This function takes either a List of integers or a string representation of a list
    and returns the sum of the numbers.

    Args:
        num_list: List[int] or str - Either a list of integers or a string representing a list
            e.g. "[1, 2, 3]" or [1, 2, 3]

    Returns:
        int: The sum of all numbers in the list

    Raises:
        ValueError: If the string cannot be converted to a list of integers
        SyntaxError: If the string is not properly formatted
    """
    if isinstance(num_list, str):
        try:
            num_list = ast.literal_eval(num_list)
            if not isinstance(num_list, list):
                raise ValueError("String must represent a list")
        except (ValueError, SyntaxError) as e:
            raise ValueError(f"Invalid input string format: {e}")

    # Verify all elements are integers
    if not all(isinstance(x, int) for x in num_list):
        raise ValueError("All elements must be integers")

    return sum(num_list)

schema = function_to_schema(add_numbers)
print(json.dumps(schema, indent=2))

{
  "type": "function",
  "function": {
    "name": "add_numbers",
    "description": "This function takes either a List of integers or a string representation of a list\n    and returns the sum of the numbers.\n\n    Args:\n        num_list: List[int] or str - Either a list of integers or a string representing a list\n            e.g. \"[1, 2, 3]\" or [1, 2, 3]\n\n    Returns:\n        int: The sum of all numbers in the list\n\n    Raises:\n        ValueError: If the string cannot be converted to a list of integers\n        SyntaxError: If the string is not properly formatted",
    "parameters": {
      "type": "object",
      "properties": {
        "num_list": {
          "type": "string"
        }
      },
      "required": [
        "num_list"
      ]
    }
  }
}


In [8]:
from typing import List, Union
import ast

def multiply_numbers(num_list: Union[List[int], str]) -> int:
    """
    This function takes either a List of integers or a string representation of a list
    and returns the product of all numbers.

    Args:
        num_list: List[int] or str - Either a list of integers or a string representing a list
            e.g. "[1, 2, 3]" or [1, 2, 3]

    Returns:
        int: The product of all numbers in the list

    Raises:
        ValueError: If the string cannot be converted to a list of integers,
                   if the list is empty, or if any element is not an integer
        SyntaxError: If the string is not properly formatted
    """
    # Handle string input
    if isinstance(num_list, str):
        try:
            num_list = ast.literal_eval(num_list)
            if not isinstance(num_list, list):
                raise ValueError("String must represent a list")
        except (ValueError, SyntaxError) as e:
            raise ValueError(f"Invalid input string format: {e}")

    # Check if list is empty
    if not num_list:
        raise ValueError("List cannot be empty")

    # Verify all elements are integers
    if not all(isinstance(x, int) for x in num_list):
        raise ValueError("All elements must be integers")

    # Calculate product using reduce and multiplication operator
    from functools import reduce
    from operator import mul
    return reduce(mul, num_list)

In [11]:
tools = [add_numbers, multiply_numbers]
tool_schemas = [function_to_schema(tool) for tool in tools]

response = run_llm(
    content = """
    [23,51,321]
    """,
    system_message = """
    Use the appropriate tool to calculate the sum of numbers, and only the tool and nothing else.
    """,
    tool_schemas = tool_schemas
)

print(response)

[ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ym9A1oUPgs4HAz9OMHVVp6RB', function=Function(arguments='{"num_list":"[23,51,321]"}', name='add_numbers'), type='function')])]


In [ ]:
response

[ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_enJSWBrayTgSlFCKgrgw6BGz', function=Function(arguments='{"num_list":"[23,51,321]"}', name='add_numbers'), type='function')])]

In [9]:
tools_map = {tool.__name__: tool for tool in tools}
messages = []

def execute_tool_call(tool_call, tools_map):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")

    # call corresponding function with provided arguments
    return tools_map[name](**args)

for tool_call in response[0].tool_calls:
            result = execute_tool_call(tool_call, tools_map)

            # add result back to conversation
            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result,
            }
            messages.append(result_message)

NameError: name 'tools' is not defined

In [10]:
def run_agent(system_message, tools, messages):

    num_init_messages = len(messages)
    messages = messages.copy()

    while True:

        # turn python functions into tools and save a reverse map
        tool_schemas = [function_to_schema(tool) for tool in tools]
        tools_map = {tool.__name__: tool for tool in tools}

        # === 1. get openai completion ===
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0,
            messages=[{"role": "system", "content": system_message}] + messages,
            tools=tool_schemas or None,
        )
        message = response.choices[0].message
        messages.append(message)

        if message.content:  # print assistant response
            print("Assistant:", message.content)

        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools_map)

            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result),
            }
            messages.append(result_message)

    # ==== 3. return new messages =====
    return messages[num_init_messages:]


def execute_tool_call(tool_call, tools_map):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")

    # call corresponding function with provided arguments
    return str(tools_map[name](**args))


tools = [add_numbers]
messages = []
system_message = """
You are an expert number processor and classifier. Your task is to extract and sum only the numbers from any input, ignoring all non-numeric values.

Rules:
1. Only process numeric values (integers)
2. Ignore all non-numeric values (strings, letters, special characters)
3. Use the add_numbers function to calculate the sum
4. Format the input properly before passing to add_numbers

Examples:
Input: <input>["hello", 42, "pizza", 2, 5]</input>
Process: Extract numbers [42, 2, 5]
Output: 49

Input: <input>[asj,cg,111,42,2]</input>
Process: Extract numbers [111, 42, 2]
Output: 155

Input: <input>[text, more, 100, words, 50]</input>
Process: Extract numbers [100, 50]
Output: 150

For any input, first extract the numbers, then use add_numbers function to calculate their sum.
Make sure to format the input as a proper list string with square brackets before passing to add_numbers.
"""


while True:
    user = input("User: ")
    messages.append({"role": "user", "content": user})

    new_messages = run_agent(system_message, tools, messages)
    messages.extend(new_messages)

KeyboardInterrupt: Interrupted by user

In [11]:
class Agent(BaseModel):
    name: str = "Agent"
    llm: str = "gpt-4o-mini"
    system_message: str = "You are a helpful Agent"
    tools: list = []

In [12]:
def run_agent(agent, messages):

    num_init_messages = len(messages)
    messages = messages.copy()

    while True:

        # turn python functions into tools and save a reverse map
        tool_schemas = [function_to_schema(tool) for tool in agent.tools]
        tools_map = {tool.__name__: tool for tool in agent.tools}

        # === 1. get openai completion ===
        response = client.chat.completions.create(
            model=agent.llm,
            messages=[{"role": "system", "content": agent.system_message}] + messages,
            tools=tool_schemas or None,
        )
        message = response.choices[0].message
        messages.append(message)

        if message.content:  # print assistant response
            print("Assistant:", message.content)

        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools_map)

            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result),
            }
            messages.append(result_message)

    # ==== 3. return new messages =====
    return messages[num_init_messages:]


def execute_tool_call(tool_call, tools_map):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")

    # call corresponding function with provided arguments
    return str(tools_map[name](**args))

In [ ]:
calculator_add = Agent(
    name="Addition Calculator",
    system_message="""You are an expert number processor. Extract and sum only the numbers from any input, ignoring non-numeric values.
    Example:
    Input: [text, 100, words, 50]
    Process: Extract numbers [100, 50]
    Output: 150""",
    tools=[add_numbers],
)

calculator_multiply = Agent(
    name="Multiplication Calculator",
    system_message="""You are an expert number processor. Extract and multiply only the numbers from any input, ignoring non-numeric values.
    Example:
    Input: [text, 4, words, 5]
    Process: Extract numbers [4, 5]
    Output: 20""",
    tools=[multiply_numbers],
)

messages = []
user_query = "[hello, 10, world, 5, test, 2]"
print("User:", user_query)
messages.append({"role": "user", "content": user_query})

response = run_agent(calculator_add, messages)  # Addition calculator
messages.extend(response)

user_query = "Now multiply these numbers"  # implicitly refers to the numbers from previous input
print("User:", user_query)
messages.append({"role": "user", "content": user_query})
response = run_agent(calculator_multiply, messages)  # Multiplication calculator

User: [hello, 10, world, 5, test, 2]
Assistant: add_numbers({'num_list': '[10, 5, 2]'})
Assistant: The sum of the numbers extracted from the input is 17.
User: Now multiply these numbers
Assistant: multiply_numbers({'num_list': '[10, 5, 2]'})
Assistant: The product of the numbers extracted from the input is 100.


In [13]:
class Response(BaseModel):
    agent: Optional[Agent]
    messages: list

In [14]:
def run_agent(agent, messages):

    current_agent = agent
    num_init_messages = len(messages)
    messages = messages.copy()

    while True:

        # turn python functions into tools and save a reverse map
        tool_schemas = [function_to_schema(tool) for tool in current_agent.tools]
        tools = {tool.__name__: tool for tool in current_agent.tools}

        # === 1. get openai completion ===
        response = client.chat.completions.create(
            model=agent.llm,
            messages=[{"role": "system", "content": current_agent.system_message}]
            + messages,
            tools=tool_schemas or None,
        )
        message = response.choices[0].message
        messages.append(message)

        if message.content:  # print agent response
            print(f"{current_agent.name}:", message.content)

        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools, current_agent.name)

            if type(result) is Agent:  # if agent transfer, update current agent
                current_agent = result
                result = (
                    f"Transfered to {current_agent.name}. Adopt persona immediately."
                )

            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result),
            }
            messages.append(result_message)

    # ==== 3. return last agent used and new messages =====
    return Response(agent=current_agent, messages=messages[num_init_messages:])


def execute_tool_call(tool_call, tools, agent_name):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"{agent_name}:", f"{name}({args})")

    return str(tools[name](**args))  # call corresponding function with provided arguments

## ReAct

In [ ]:
def add_numbers(a: float, b: float) -> float:
    """
    Add two numbers together.

    Args:
        a: First number
        b: Second number

    Returns:
        The sum of a and b
    """
    return a + b

def subtract_numbers(a: float, b: float) -> float:
    """
    Subtract b from a.

    Args:
        a: First number
        b: Second number

    Returns:
        The result of a - b
    """
    return a - b

def multiply_numbers(a: float, b: float) -> float:
    """
    Multiply two numbers together.

    Args:
        a: First number
        b: Second number

    Returns:
        The product of a and b
    """
    return a * b

def divide_numbers(a: float, b: float) -> float:
    """
    Divide a by b.

    Args:
        a: First number (dividend)
        b: Second number (divisor)

    Returns:
        The result of a / b

    Raises:
        ValueError: If b is zero
    """
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

In [ ]:
react_system_message = """You are a mathematical reasoning agent that follows the ReAct pattern: Thought, Action, Observation.

For each step of your reasoning:
1. Thought: First explain your thinking
2. Action: Then choose and use a tool
3. Observation: Observe the result

Format your responses as:
Thought: <your reasoning>
Action: <tool_name>(<parameters>)
Observation: <result>
Thought: <your next step>
...

Available tools:
- add_numbers(a, b): Add two numbers
- subtract_numbers(a, b): Subtract b from a
- multiply_numbers(a, b): Multiply two numbers
- divide_numbers(a, b): Divide a by b

Always break down complex calculations into steps using this format."""

react_agent = Agent(
    name="ReActMath",
    llm="gpt-4o-mini",
    system_message=react_system_message,
    tools=[add_numbers, subtract_numbers, multiply_numbers, divide_numbers]
)

In [ ]:
# Example usage
messages = [{
    "role": "user",
    "content": "Calculate (23 + 7) * 3 - 15"
}]

response = run_agent(react_agent, messages)

ReActMath: Thought: First, I need to calculate the sum of 23 and 7. Then I will multiply the result by 3, and finally, I will subtract 15 from that product. I'll break this down into steps for clarity. 

Action: I will first add 23 and 7. 
functions.add_numbers({ a: 23, b: 7 })

Observation: Let's perform the addition.
ReActMath: add_numbers({'a': 23, 'b': 7})
ReActMath: Thought: The sum of 23 and 7 is 30. Now, I will multiply this result by 3. 

Action: I will multiply 30 by 3. 
functions.multiply_numbers({ a: 30, b: 3 }) 

Observation: Let's perform the multiplication.
ReActMath: multiply_numbers({'a': 30, 'b': 3})
ReActMath: Thought: The product of 30 and 3 is 90. Now, I need to subtract 15 from this result. 

Action: I will subtract 15 from 90. 
functions.subtract_numbers({ a: 90, b: 15 }) 

Observation: Let's perform the subtraction.
ReActMath: subtract_numbers({'a': 90, 'b': 15})
ReActMath: Thought: The result of subtracting 15 from 90 is 75. Therefore, the final result of the ca

## Agentic RAG

In [ ]:
import requests
from bs4 import BeautifulSoup
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Dict
import re

class RAGAgent:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.embedder = SentenceTransformer(model_name)
        self.documents = []
        self.index = None
        self.dimension = 384  # Default for MiniLM-L6

    def retrieve_documents(self, url: str):
        """Retrieval tool to get and process documents"""
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Get main content from Wikipedia
        content = soup.find(id="mw-content-text")
        if content:
            paragraphs = content.find_all('p')
            self.documents = [p.text for p in paragraphs if len(p.text.split()) > 20]

        return "Documents retrieved successfully"

    def create_index(self):
        """Create FAISS index from documents"""
        embeddings = self.embedder.encode(self.documents)
        self.index = faiss.IndexFlatL2(self.dimension)
        self.index.add(np.array(embeddings).astype('float32'))

    def check_relevance(self, query: str, k: int = 3) -> List[str]:
        """Check relevance of query against documents"""
        query_vector = self.embedder.encode([query])
        D, I = self.index.search(np.array(query_vector).astype('float32'), k)
        return [self.documents[i] for i in I[0]]

    def rewrite_query(self, query: str, context: List[str]) -> str:
        """Rewrite the query based on retrieved context"""
        # This would use the OpenAI API to rewrite the query
        # For now, we'll return the original query
        return query

    def generate_answer(self, query: str, context: List[str]) -> str:
        """Generate answer based on context"""
        # This would use the OpenAI API to generate the answer
        # For now, we'll return the most relevant context
        return context[0] if context else "No relevant information found."

In [ ]:
from typing import List, Dict
import requests
from bs4 import BeautifulSoup
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

class DocumentStore:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.embedder = SentenceTransformer(model_name)
        self.documents = []
        self.index = None
        self.dimension = 384  # Default for MiniLM-L6

    def add_documents(self, url: str) -> str:
        """Retrieval tool to get and process documents"""
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Get main content from Wikipedia
        content = soup.find(id="mw-content-text")
        if content:
            paragraphs = content.find_all('p')
            self.documents = [p.text for p in paragraphs if len(p.text.split()) > 20]

        # Create embeddings and index
        embeddings = self.embedder.encode(self.documents)
        self.index = faiss.IndexFlatL2(self.dimension)
        self.index.add(np.array(embeddings).astype('float32'))
        return f"Processed {len(self.documents)} documents"

    def search(self, query: str, k: int = 3) -> List[str]:
        """Search for relevant documents"""
        query_vector = self.embedder.encode([query])
        D, I = self.index.search(np.array(query_vector).astype('float32'), k)
        return [self.documents[i] for i in I[0]]

def retrieve_documents(url: str):
    """Tool function to retrieve and index documents"""
    return doc_store.add_documents(url)

def search_documents(query: str):
    """Tool function to search documents"""
    return doc_store.search(query)

# Initialize document store globally
doc_store = DocumentStore()

# Create our agents
retrieval_agent = Agent(
    name="Retrieval Agent",
    system_message="""You are a document retrieval agent.
    Your task is to retrieve and index documents from provided URLs.
    After indexing, you should confirm success and suggest asking questions about the content.""",
    tools=[retrieve_documents],
    model="gpt-3.5-turbo"
)

qa_agent = Agent(
    name="QA Agent",
    system_message="""You are a question answering agent.
    Using the provided context from relevant documents, answer the user's questions.
    Always cite specific parts of the context in your answers.
    If you can't find relevant information in the context, say so.""",
    tools=[search_documents],
    model="gpt-3.5-turbo"
)

# Main execution
messages = []
current_agent = retrieval_agent

# First, let's index Alan Turing's Wikipedia article
user_query = "Please retrieve and index this URL: https://en.wikipedia.org/wiki/Alan_Turing"
messages.append({"role": "user", "content": user_query})
response = run_agent(current_agent, messages)
messages.extend(response)

# Switch to QA agent for questions
current_agent = qa_agent

while True:
    user = input("Ask a question about Alan Turing (or 'quit' to exit): ")
    if user.lower() == 'quit':
        break

    messages.append({"role": "user", "content": user})
    response = run_agent(current_agent, messages)
    messages.extend(response)

In [20]:
!pip install sentence-transformers
!pip install faiss-cpu
!pip install beautifulsoup4
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [27]:
import requests
from bs4 import BeautifulSoup
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Dict
import json

class DocumentStore:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.embedder = SentenceTransformer(model_name)
        self.documents = []
        self.index = None
        self.dimension = 384

    def add_documents(self, url: str) -> str:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        content = soup.find(id="mw-content-text")
        if content:
            paragraphs = content.find_all('p')
            self.documents = [p.text for p in paragraphs if len(p.text.split()) > 20]

        embeddings = self.embedder.encode(self.documents)
        self.index = faiss.IndexFlatL2(self.dimension)
        self.index.add(np.array(embeddings).astype('float32'))
        return f"Processed {len(self.documents)} documents"

    def search(self, query: str, k: int = 3) -> List[str]:
        query_vector = self.embedder.encode([query])
        D, I = self.index.search(np.array(query_vector).astype('float32'), k)
        return [self.documents[i] for i in I[0]]

# Initialize document store
doc_store = DocumentStore()

def retrieve_documents(url: str):
    """Tool function for document retrieval"""
    return doc_store.add_documents(url)

def search_context(query: str):
    """Tool function for searching documents"""
    return doc_store.search(query)

def check_relevance(context: List[str]) -> bool:
    """
    Tool function to check if retrieved context is relevant using an LLM.

    Args:
        context: List of context strings to evaluate

    Returns:
        bool: True if context is relevant, False otherwise
    """
    if not context:
        return False

    system_message = """You are a relevance checking assistant.
    Evaluate if the given context is relevant and substantial enough to answer questions.
    Return only 'true' or 'false'."""

    prompt = f"""Evaluate if this context is relevant and substantial (contains meaningful information):

Context: {context}

Return only 'true' or 'false'."""

    messages = run_llm(
        content=prompt,
        system_message=system_message
    )

    # Get the last message which contains the LLM's response
    result = messages[-1].content.lower().strip()
    return result == 'true'

def rewrite_query(query: str, context: List[str]) -> str:
    """
    Tool function to rewrite a query based on context using an LLM.

    Args:
        query: Original query to rewrite
        context: List of context strings to use for rewriting

    Returns:
        str: Rewritten query incorporating context
    """
    system_message = """You are a query rewriting assistant.
    Your task is to rewrite the original query to incorporate relevant context.
    Maintain the original intent while making it more specific based on the context."""

    prompt = f"""Original Query: {query}

Available Context: {context}

Rewrite the query to be more specific using the context.
Maintain the original intent but make it more precise."""

    messages = run_llm(
        content=prompt,
        system_message=system_message
    )

    # Get the last message which contains the rewritten query
    return messages[-1].content.strip()

rag_agent = Agent(
    name="RAG Agent",
    system_message="""You are an intelligent RAG agent that follows a specific workflow:
    1. First, determine if you need to retrieve documents
    2. If yes, use the retrieval tool
    3. Check the relevance of retrieved documents
    4. Either rewrite the query or generate an answer
    5. Always cite your sources from the context

    Be explicit about each step you're taking.""",
    tools=[retrieve_documents, search_context, check_relevance, rewrite_query],
    llm="gpt-4o-mini"
)

def run_rag_agent(agent, messages):
    num_init_messages = len(messages)
    messages = messages.copy()

    while True:
        # Get tool schemas and tool map
        tool_schemas = [function_to_schema(tool) for tool in agent.tools]
        tools = {tool.__name__: tool for tool in agent.tools}

        # Make API call with system message and history
        response = client.chat.completions.create(
            model=agent.llm,
            messages=[{"role": "system", "content": agent.system_message}] + messages,
            tools=tool_schemas,
        )

        # Get and append the assistant's message
        message = response.choices[0].message
        messages.append({
            "role": "assistant",
            "content": message.content,
            "tool_calls": message.tool_calls if hasattr(message, "tool_calls") else None
        })

        if message.content:
            print(f"{agent.name}:", message.content)

        if not hasattr(message, "tool_calls") or not message.tool_calls:
            break

        # Handle tool calls
        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools, agent.name)

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result),
            })

    return messages[num_init_messages:]

# Main execution
messages = []

# First, index a document
url = "https://en.wikipedia.org/wiki/Alan_Turing"
messages.append({
    "role": "user",
    "content": f"Please retrieve and index this document: {url}"
})

while True:
    try:
        response = run_rag_agent(rag_agent, messages)
        messages.extend(response)

        user_input = input("\nUser (type 'quit' to exit): ")
        if user_input.lower() == 'quit':
            break

        messages.append({"role": "user", "content": user_input})

    except Exception as e:
        print(f"Error occurred: {e}")
        break

RAG Agent: retrieve_documents({'url': 'https://en.wikipedia.org/wiki/Alan_Turing'})
RAG Agent: I have retrieved and processed the document from the provided URL. It contains detailed information about Alan Turing, including his contributions to mathematics, computing, cryptography, and artificial intelligence, as well as his personal life and legacy. 

If you have a specific query or need information from this document, please let me know!

User (type 'quit' to exit): who was alan turning?
RAG Agent: check_relevance({'context': 'Alan Turing was an English mathematician, logician, cryptanalyst, and computer scientist who is widely considered to be the father of computer science. He made significant contributions to the development of theoretical computer science, including the concept of the Turing machine, which is a foundational model of computation. Turing also played a crucial role in breaking the Enigma code during World War II, which was vital for the Allied war effort. Beyond his

KeyboardInterrupt: Interrupted by user

## Supervisor + Workers

In [23]:
from typing import List, Union
import ast
from functools import reduce
from operator import mul

def add_numbers(num_list: Union[List[int], str]) -> int:
    """
    Add a list of numbers together.
    Accepts either a list of integers or a string representation of a list.

    Args:
        num_list: List[int] or str - Either a list of integers or a string like "[1,2,3]"

    Returns:
        int: Sum of all numbers

    Raises:
        ValueError: If input format is invalid or no valid integers found
    """
    # Handle string input
    if isinstance(num_list, str):
        try:
            # Clean the input string
            cleaned_str = num_list.strip()
            if not (cleaned_str.startswith('[') and cleaned_str.endswith(']')):
                raise ValueError("Input must be enclosed in square brackets")

            # Parse numbers from string
            items = cleaned_str[1:-1].split(',')
            processed_list = []
            for item in items:
                try:
                    # Only process valid integers
                    if item.strip().isdigit():
                        processed_list.append(int(item.strip()))
                except ValueError:
                    continue
            num_list = processed_list
        except Exception as e:
            raise ValueError(f"Invalid input string format: {e}")

    # Filter and validate numbers
    numbers = [x for x in num_list if isinstance(x, int)]
    if not numbers:
        raise ValueError("No valid integers found in the input")

    return sum(numbers)

def multiply_numbers(num_list: Union[List[int], str]) -> int:
    """
    Multiply a list of numbers together.
    Accepts either a list of integers or a string representation of a list.

    Args:
        num_list: List[int] or str - Either a list of integers or a string like "[2,3,4]"

    Returns:
        int: Product of all numbers

    Raises:
        ValueError: If input format is invalid or no valid integers found
    """
    # Handle string input
    if isinstance(num_list, str):
        try:
            # Clean the input string
            cleaned_str = num_list.strip()
            if not (cleaned_str.startswith('[') and cleaned_str.endswith(']')):
                raise ValueError("Input must be enclosed in square brackets")

            # Parse numbers from string
            items = cleaned_str[1:-1].split(',')
            processed_list = []
            for item in items:
                try:
                    # Only process valid integers
                    if item.strip().isdigit():
                        processed_list.append(int(item.strip()))
                except ValueError:
                    continue
            num_list = processed_list
        except Exception as e:
            raise ValueError(f"Invalid input string format: {e}")

    # Filter and validate numbers
    numbers = [x for x in num_list if isinstance(x, int)]
    if not numbers:
        raise ValueError("No valid integers found in the input")

    return reduce(mul, numbers)

def transfer_to_addition() -> Agent:
    """
    Transfer control to the addition calculator agent.

    Returns:
        str: The addition calculator agent
    """
    return "transfer_to_addition"

def transfer_to_multiplication() -> Agent:
    """
    Transfer control to the multiplication calculator agent.

    Returns:
        str: The multiplication calculator agent
    """
    return "transfer_to_multiplication"

def transfer_to_triage() -> Agent:
    """
    Transfer control back to the main calculator triage agent.

    Returns:
        str: The calculator triage agent
    """
    return "transfer_to_triage"

# Helper function to extract numbers from text
def extract_numbers(text: str) -> List[int]:
    """
    Extract numbers from a text string.

    Args:
        text: str - Input text containing numbers

    Returns:
        List[int]: List of extracted numbers
    """
    import re
    return [int(num) for num in re.findall(r'\d+', text)]

In [24]:
class Response:
    def __init__(self, agent, messages):
        self.agent = agent
        self.messages = messages

def run_full_turn(agent, messages):
    # Get initial response from current agent
    agent_response = run_agent(agent, messages)
    new_messages = agent_response.messages
    last_message = new_messages[-1] if new_messages else None

    # Check for transfers and execute next agent if needed
    if last_message and hasattr(last_message, 'content') and last_message.content:
        # Update messages with the transfer message
        messages.extend(new_messages)

        # Handle transfers
        next_agent = None
        if "transfer_to_addition" in last_message.content:
            next_agent = addition_agent
        elif "transfer_to_multiplication" in last_message.content:
            next_agent = multiplication_agent
        elif "transfer_to_triage" in last_message.content:
            next_agent = calculator_triage_agent

        # If we have a transfer, run the next agent
        if next_agent:
            next_response = run_agent(next_agent, messages)
            return Response(next_agent, next_response.messages)

    # If no transfer, return original response
    return Response(agent_response.agent, new_messages)

# Update the agents' system messages to be more explicit
calculator_triage_agent = Agent(
    name="Calculator Triage",
    system_message=(
        "You are a calculator assistant. If user wants to:\n"
        "- Add numbers: Use transfer_to_addition()\n"
        "- Multiply numbers: Use transfer_to_multiplication()\n"
        "When you identify the operation, immediately call the appropriate transfer function."
    ),
    tools=[transfer_to_addition, transfer_to_multiplication],
    model="gpt-4o"
)

addition_agent = Agent(
    name="Addition Calculator",
    system_message=(
        "You are an addition calculator.\n"
        "1. Extract all numbers from the input\n"
        "2. Format them as a list with square brackets\n"
        "3. Use add_numbers() with the formatted list\n"
        "4. Share the result clearly\n"
        "Example: For 'add 1,2,3' → use add_numbers('[1,2,3]')"
    ),
    tools=[add_numbers, transfer_to_triage],
    model="gpt-4o"
)

multiplication_agent = Agent(
    name="Multiplication Calculator",
    system_message=(
        "You are a multiplication calculator.\n"
        "1. Extract all numbers from the input\n"
        "2. Format them as a list with square brackets\n"
        "3. Use multiply_numbers() with the formatted list\n"
        "4. Share the result clearly\n"
        "Example: For 'multiply 2,3,4' → use multiply_numbers('[2,3,4]')"
    ),
    tools=[multiply_numbers, transfer_to_triage],
    model="gpt-4o"
)

# Main loop
agent = calculator_triage_agent
messages = []

while True:
    try:
        user = input("User: ")
        if user.lower() in ['quit', 'exit', 'bye']:
            print("Goodbye!")
            break

        messages.append({"role": "user", "content": user})
        response = run_full_turn(agent, messages)
        agent = response.agent
        messages.extend(response.messages)

    except Exception as e:
        print(f"Error: {str(e)}")
        print("Resetting conversation...")
        messages = []
        agent = calculator_triage_agent

User: what can you do?
Calculator Triage: I can assist you with performing mathematical calculations, specifically addition and multiplication. If you need help with either of these operations, just let me know, and I will transfer you to the appropriate calculator.
User: what is the sum of 21 and 22
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_addition({})
Calculator Triage: transfer_to_additio

KeyboardInterrupt: 

In [26]:
# Math Genius Tools
def solve_equation(equation: str) -> str:
    """
    Solves mathematical equations.
    Args:
        equation: str - A mathematical equation to solve
            e.g. "2x + 5 = 13" or "integrate(x^2)"
    Returns:
        str: The solution to the equation
    """
    # In a real implementation, this would use a math solving library
    return f"Solution for equation: {equation}"

def perform_statistics(data: str) -> str:
    """
    Performs statistical analysis on given data.
    Args:
        data: str - Data in format "[1,2,3,4]" or "1,2,3,4"
    Returns:
        str: Statistical analysis including mean, median, mode, std dev
    """
    # In a real implementation, this would use numpy/pandas
    return f"Statistical analysis of: {data}"

# Code Writer Tools
def generate_code(specification: str, language: str) -> str:
    """
    Generates code based on specifications.
    Args:
        specification: str - Description of what the code should do
        language: str - Programming language to use
    Returns:
        str: Generated code
    """
    return f"Generated {language} code for: {specification}"

def review_code(code: str) -> str:
    """
    Reviews code for best practices and potential issues.
    Args:
        code: str - Code to review
    Returns:
        str: Code review comments
    """
    return f"Code review for: {code}"

def transfer_to_math_genius():
    """Transfer to math genius agent for complex calculations."""
    return math_genius_agent

def transfer_to_code_writer():
    """Transfer to code writer agent for programming tasks."""
    return code_writer_agent

def transfer_to_supervisor():
    """Transfer back to supervisor agent."""
    return supervisor_agent

# Agent Definitions
math_genius_agent = Agent(
    name="Math Genius",
    llm="gpt-4o",
    system_message=(
        "You are a mathematical genius capable of solving complex equations "
        "and performing advanced statistical analysis. Always show your work "
        "step by step. If a task is outside your mathematical expertise, "
        "transfer back to the supervisor."
    ),
    tools=[solve_equation, perform_statistics, transfer_to_supervisor]
)

code_writer_agent = Agent(
    name="Code Writer",
    llm="gpt-4o",
    system_message=(
        "You are an expert programmer capable of writing efficient, clean code "
        "and providing detailed code reviews. Always explain your code and "
        "include comments. If a task is outside your programming expertise, "
        "transfer back to the supervisor."
    ),
    tools=[generate_code, review_code, transfer_to_supervisor]
)

supervisor_agent = Agent(
    name="Supervisor",
    llm="gpt-4o",
    system_message=(
        "You are a supervisor agent responsible for delegating tasks to specialized agents. "
        "For mathematical problems, delegate to the Math Genius. "
        "For programming tasks, delegate to the Code Writer. "
        "Ensure all responses are complete and coordinate between agents when needed."
    ),
    tools=[transfer_to_math_genius, transfer_to_code_writer]
)

# Example usage
def process_request(user_input: str):
    """
    Process user request through the multi-agent system.
    Args:
        user_input: str - User's question or request
    Returns:
        List of messages containing the conversation
    """
    messages = [{"role": "user", "content": user_input}]
    response = run_full_turn(supervisor_agent, messages)
    return response.messages

# Interactive testing loop
if __name__ == "__main__":
    print("Welcome to the Multi-Agent System!")
    print("You can interact with a supervisor who will delegate to either:")
    print("1. Math Genius - for mathematical problems")
    print("2. Code Writer - for programming tasks")
    print("Type 'exit', 'quit', or 'bye' to end the conversation\n")

    agent = supervisor_agent
    messages = []

    while True:
        try:
            user_input = input("User: ")
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("Goodbye!")
                break

            messages.append({"role": "user", "content": user_input})
            response = run_full_turn(agent, messages)
            agent = response.agent
            messages.extend(response.messages)

        except Exception as e:
            print(f"Error: {str(e)}")
            print("Resetting conversation...")
            messages = []
            agent = supervisor_agent

Welcome to the Multi-Agent System!
You can interact with a supervisor who will delegate to either:
1. Math Genius - for mathematical problems
2. Code Writer - for programming tasks
Type 'exit', 'quit', or 'bye' to end the conversation

User: what is the quadratic equation
Supervisor: The quadratic equation is a polynomial equation of the second degree, typically in the form:

\[ ax^2 + bx + c = 0 \]

where:
- \( x \) represents an unknown variable,
- \( a \), \( b \), and \( c \) are coefficients with \( a \neq 0 \).

The solutions to the quadratic equation can be found using the quadratic formula:

\[ x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a} \]

This formula provides the values of \( x \) that satisfy the equation, where the discriminant \( b^2 - 4ac \) determines the nature of the roots (real and distinct, real and repeated, or complex).
User: What is the python code to calculate sum of n fibonachi numbers, ask the code expert
Supervisor: transfer_to_code_writer({})
Supervisor: To calc

KeyboardInterrupt: Interrupted by user